<a href="https://colab.research.google.com/github/JAK0707/Job-recommendation-model/blob/main/Job_recommendation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas scikit-learn

In [3]:
import pandas as pd
import re

# Load the dataset
from google.colab import files
uploaded = files.upload()

file_name = 'job_skills.csv'
data = pd.read_csv(file_name)

Saving job_skills.csv to job_skills.csv


In [4]:
# Function to clean text
def clean_text(text):
    if isinstance(text, str):
        # Convert text to lowercase
        text = text.lower()
        # Remove special characters and numbers
        text = re.sub(r'[^a-z\s]', '', text)
        return text
    return ''

In [5]:
# Apply text cleaning to relevant columns
data['Responsibilities'] = data['Responsibilities'].apply(clean_text)
data['Minimum Qualifications'] = data['Minimum Qualifications'].apply(clean_text)
data['Preferred Qualifications'] = data['Preferred Qualifications'].apply(clean_text)

In [6]:
# Step 3: Extract Skills
# Define a list of common skills
common_skills = [
    "python", "java", "c++", "javascript", "sql", "machine learning",
    "deep learning", "data analysis", "project management", "communication",
    "leadership", "teamwork", "problem-solving", "cloud computing",
    "networking", "security", "software development", "data science",
    "engineering", "mathematics", "statistics", "artificial intelligence"
]

def extract_skills(text):
    if isinstance(text, str):
        found_skills = [skill for skill in common_skills if skill in text]
        return ', '.join(found_skills)
    return ''

In [7]:
# Apply skill extraction to relevant columns
data['Skills_Responsibilities'] = data['Responsibilities'].apply(extract_skills)
data['Skills_Minimum'] = data['Minimum Qualifications'].apply(extract_skills)
data['Skills_Preferred'] = data['Preferred Qualifications'].apply(extract_skills)

In [8]:
# Combine all extracted skills into one column
data['All_Skills'] = data[['Skills_Responsibilities', 'Skills_Minimum', 'Skills_Preferred']].apply(lambda x: ', '.join(set(', '.join(x).split(', '))), axis=1)

In [9]:
# Step 4: Train a Model
# Feature Engineering
# Here we will convert the skills into a binary feature vector for model training
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '), binary=True)
X = vectorizer.fit_transform(data['All_Skills'])

# For demonstration purposes, we'll use job titles as labels (you can change this)
y = data['Title']

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
# Train a Model
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [11]:

# Step 5: Implement Resume Parsing and Matching
# Function to parse resume and extract skills
def parse_resume(resume_text):
    resume_text = clean_text(resume_text)
    found_skills = [skill for skill in common_skills if skill in resume_text]
    return ', '.join(found_skills)

# Function to match resume skills with job listing skills
def match_skills(resume_skills, job_skills):
    resume_skills_set = set(resume_skills.split(', '))
    job_skills_set = set(job_skills.split(', '))
    matched_skills = resume_skills_set.intersection(job_skills_set)
    return ', '.join(matched_skills), len(matched_skills)

# Function to get skills required by a specific company
def get_company_skills(company_name):
    company_jobs = data[data['Company'].str.lower() == company_name.lower()]
    all_skills = ', '.join(company_jobs['All_Skills'])
    unique_skills = set(all_skills.split(', '))
    return ', '.join(unique_skills)

In [12]:
# Step 4: Train a Model
# Feature Engineering
# Here we will convert the skills into a binary feature vector for model training
from sklearn.feature_extraction.text import CountVectorizer

def custom_tokenizer(text):
    return text.split(', ')

vectorizer = CountVectorizer(tokenizer=custom_tokenizer, binary=True)
X = vectorizer.fit_transform(data['All_Skills'])

# For demonstration purposes, we'll use job titles as labels (you can change this)
y = data['Title']

# ... rest of your code ...

# Save the model
import joblib
joblib.dump(model, 'job_matching_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl') # This should now work without errors

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['vectorizer.pkl']

In [13]:
# Example Resume Text (for demonstration purposes)
resume_text_example = """
Experienced software developer with expertise in python, java, and machine learning.
Proven ability to manage projects and communicate effectively within a team environment.
"""

# Extract skills from the example resume text
resume_skills = parse_resume(resume_text_example)

# Match skills with a job listing (for demonstration, using the first job listing in the dataset)
matched_skills, num_matched_skills = match_skills(resume_skills, data['All_Skills'].iloc[0])
print(f'Resume Skills: {resume_skills}')
print(f'Matched Skills: {matched_skills} ({num_matched_skills})')

# Get skills required by a specific company (e.g., Google)
company_skills = get_company_skills('Google')
print(f'Skills required by Google: {company_skills}')


Resume Skills: python, java, machine learning
Matched Skills:  (0)
Skills required by Google: , machine learning, java, project management, data analysis, statistics, cloud computing, leadership, javascript, python, security, engineering, networking, software development, data science, communication, sql, artificial intelligence, mathematics, teamwork


In [14]:
def match_skills(resume_skills, job_skills):
    resume_skills_set = set(resume_skills.split(', '))
    job_skills_set = set(job_skills.split(', '))
    matched_skills = resume_skills_set.intersection(job_skills_set)
    return ', '.join(matched_skills), len(matched_skills)

In [15]:
# Example Resume Text (for demonstration purposes)
resume_text_example = """
Experienced software developer with expertise in Python, Java,and machine learning.
Proven ability to manage projects and communicate effectively within a team environment.
"""

# Extract skills from the example resume text
resume_skills = parse_resume(resume_text_example)

# Match skills with a job listing (for demonstration, using the first job listing in the dataset)
job_listing_skills = data['All_Skills'].iloc[0]
matched_skills, num_matched_skills = match_skills(resume_skills, job_listing_skills)

print(f'Resume Skills: {resume_skills}')
print(f'Job Listing Skills: {job_listing_skills}')
print(f'Matched Skills: {matched_skills} ({num_matched_skills})')


Resume Skills: python, java, machine learning
Job Listing Skills: cloud computing, engineering, project management, communication
Matched Skills:  (0)


In [16]:
# Assuming the previous steps of loading the dataset and defining functions are already completed

# Define a sample resume text
resume_text_example = """
Experienced software developer with expertise in Python, Java, and machine learning.
Proven ability to manage projects and communicate effectively within a team environment.
"""

# Function to parse resume and extract skills
def parse_resume(resume_text):
    resume_text = clean_text(resume_text)
    found_skills = [skill for skill in common_skills if skill in resume_text]
    return ', '.join(found_skills)

# Extract skills from the example resume text
resume_skills = parse_resume(resume_text_example)

# Function to get skills required by a specific company
def get_company_skills(company_name):
    company_jobs = data[data['Company'].str.lower() == company_name.lower()]
    all_skills = ', '.join(company_jobs['All_Skills'])
    unique_skills = set(all_skills.split(', '))
    return ', '.join(unique_skills)

# Get skills required by Google
company_name = 'Google'
company_skills = get_company_skills(company_name)

# Match skills with the resume
matched_skills, num_matched_skills = match_skills(resume_skills, company_skills)

# Print the results
print(f'Resume Skills: {resume_skills}')
print(f'{company_name} Skills: {company_skills}')
print(f'Matched Skills: {matched_skills} ({num_matched_skills})')


Resume Skills: python, java, machine learning
Google Skills: , machine learning, java, project management, data analysis, statistics, cloud computing, leadership, javascript, python, security, engineering, networking, software development, data science, communication, sql, artificial intelligence, mathematics, teamwork
Matched Skills: java, machine learning, python (3)


In [17]:
# Assuming the previous steps of loading the dataset, defining functions, and training the model are already completed

# Example Resume Text (for demonstration purposes)
resume_text_example = """
Experienced software developer with expertise in Python, Java, and machine learning.
Proven ability to manage projects and communicate effectively within a team environment.
"""

# Function to parse resume and extract skills
def parse_resume(resume_text):
    resume_text = clean_text(resume_text)
    found_skills = [skill for skill in common_skills if skill in resume_text]
    return ', '.join(found_skills)

# Extract skills from the example resume text
resume_skills = parse_resume(resume_text_example)

# Transform resume skills using the vectorizer
resume_skills_vectorized = vectorizer.transform([resume_skills])

# Predict job titles using the trained model
predicted_job_titles = model.predict(resume_skills_vectorized)

# Get skills required by a specific company
def get_company_skills(company_name):
    company_jobs = data[data['Company'].str.lower() == company_name.lower()]
    all_skills = ', '.join(company_jobs['All_Skills'])
    unique_skills = set(all_skills.split(', '))
    return ', '.join(unique_skills)

# Get skills required by Google
company_name = 'Google'
company_skills = get_company_skills(company_name)

# Match skills with the resume
matched_skills, num_matched_skills = match_skills(resume_skills, company_skills)

# Print the results
print(f'Resume Skills: {resume_skills}')
print(f'Predicted Job Titles: {predicted_job_titles}')
print(f'{company_name} Skills: {company_skills}')
print(f'Matched Skills: {matched_skills} ({num_matched_skills})')


Resume Skills: python, java, machine learning
Predicted Job Titles: ['Strategic Cloud Engineer, Google Cloud Professional Services']
Google Skills: , machine learning, java, project management, data analysis, statistics, cloud computing, leadership, javascript, python, security, engineering, networking, software development, data science, communication, sql, artificial intelligence, mathematics, teamwork
Matched Skills: java, machine learning, python (3)


In [26]:
# Function to get open positions at a specific company
def get_open_positions(company_name):
    company_jobs = data[data['Company'].str.lower() == company_name.lower()]
    open_positions = company_jobs['Title'].unique()
    return '\n'.join(open_positions)

# Updated feature to list skills required and open positions for a specific company with line breaks
def skills_and_positions_for_company(company_name):
    company_skills = get_company_skills(company_name)
    open_positions = get_open_positions(company_name)
    return f'To get a job at {company_name}, you need the following skills: {company_skills}.\nCurrent open positions:\n{open_positions}'

# Example usage to get skills required and open positions for Google
print(skills_and_positions_for_company('Google'))


To get a job at Google, you need the following skills: , machine learning, java, project management, data analysis, statistics, cloud computing, leadership, javascript, python, security, engineering, networking, software development, data science, communication, sql, artificial intelligence, mathematics, teamwork.
Current open positions:
Google Cloud Program Manager
Supplier Development Engineer (SDE), Cable/Connector
Data Analyst, Product and Tools Operations, Google Technical Services
Developer Advocate, Partner Engineering
Program Manager, Audio Visual (AV) Deployments
Associate Account Strategist (Czech/Slovak), Global Customer Experience
Supplier Development Engineer, Camera, Consumer Hardware
Strategic Technology Partner Manager, Healthcare and Life Sciences
Manufacturing Business Manager, Google Hardware
Solutions Architect, Healthcare and Life Sciences, Google Cloud
Data Analyst, Consumer Hardware
Partner Onboarding Manager (Americas)
Associate Account Strategist (Ukrainian), G